<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/twadr_0percent_mixed_pubmed_bigtweet_twadr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'tward_training_folder_0.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"tward_validation_folder_0.csv",header=None)
valid.head()


,0,1
0,1811,cant shut up for the whole day
1,903,angry
2,1576,got me to the ceiling
3,2201,anxiety
4,416,tingling


In [12]:
# testing data set

test = pd.read_csv(path/"tward_testing_overlap_0.csv",header=None)
test.head()

,0,1
0,1576,still not tired
1,2099,can't remember the last time i slept
2,1698,i feel like
3,1811,wired
4,476,still procrastinating


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_twadr.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'was',
 'diclofenac',
 'were',
 'on',
 'is',
 'pain',
 'rt',
 'sodium',
 'by',
 'drug',
 'that',
 'as',
 'i',
 'patients',
 'at',
 'or',
 'from',
 'this',
 'you',
 'disease',
 'be',
 'mg',
 'increased',
 'group',
 'study',
 'my',
 'mood',
 'are',
 'not',
 'treatment',
 'an',
 'it',
 'after',
 'me',
 'amp',
 'have',
 'p',
 'release',
 'gluten',
 'disorder',
 ',',
 'all',
 'effect',
 'we',
 'than',
 'free',
 'abnormal',
 'withdrawal',
 'new',
 'decreased',
 'symptoms',
 'compared',
 'your',
 'more',
 'no',
 'anxiety',
 'weight',
 'day',
 'out',
 'using',
 'effects',
 'groups',
 'two',
 'up',
 'both',
 'h',
 'inflammatory',
 'time',
 'drugs',
 'our',
 'about',
 'its',
 'significantly',
 'blood',
 'between',
 'potassium',
 'acute',
 'these',
 'had',
 'anti',
 'significant',
 'used',
 'skin',
 'which',
 'like',
 'can',
 'but',
 'use',
 'disor

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.737868,4.179143,0.346916,22:58


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.019536,3.795144,0.385336,22:49


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.729517,3.749534,0.389840,23:01


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.691423,3.718100,0.393080,22:40


In [25]:
learn.save_encoder('mixed_pubmed_bigtweet_twadr_fold0overlap_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('mixed_pubmed_bigtweet_twadr_fold0overlap_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.131246,6.156239,0.104348,00:23


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.176614,4.898152,0.121739,00:26


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.674140,4.614071,0.191304,01:09


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.377638,4.325761,0.217391,01:07
1,5.164179,4.227374,0.260870,01:07


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.910917,3.999468,0.217391,01:05
1,4.706783,3.927096,0.243478,01:07


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.472293,3.780668,0.313043,01:05
1,4.325327,3.717154,0.286957,01:05


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.128567,3.523573,0.286957,01:08
1,3.987180,3.494440,0.347826,01:07


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.798090,3.453353,0.330435,01:04
1,3.646176,3.397893,0.347826,01:08


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.482186,3.331968,0.347826,01:03
1,3.388303,3.258308,0.356522,01:05
2,3.200617,3.228791,0.347826,01:08
3,3.044141,3.229238,0.330435,01:06


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.852188,3.217638,0.347826,01:10
1,2.823408,3.281309,0.330435,01:05
2,2.643030,3.229269,0.313043,01:07
3,2.490173,3.253141,0.295652,01:03


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.347867,3.216843,0.339130,01:07
1,2.316210,3.274376,0.373913,01:06
2,2.151614,3.172969,0.400000,01:07
3,1.991429,3.187582,0.391304,01:09


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.833247,3.232282,0.391304,01:09
1,1.833185,3.327243,0.391304,01:07
2,1.689593,3.277829,0.391304,01:09
3,1.550542,3.298469,0.373913,01:06


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.394125,3.330763,0.391304,01:07
1,1.401590,3.415265,0.365217,01:10
2,1.324995,3.356744,0.382609,01:04
3,1.213404,3.335640,0.400000,01:10


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.121495,3.379417,0.426087,01:10
1,1.144591,3.425703,0.417391,01:08
2,1.055923,3.489067,0.400000,01:10
3,0.949762,3.494132,0.426087,01:12


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.876389,3.600988,0.426087,01:04
1,0.901534,3.666185,0.373913,01:08
2,0.835802,3.674575,0.382609,01:05
3,0.756521,3.630085,0.382609,01:10


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.712533,3.617586,0.400000,01:09
1,0.691989,3.642014,0.373913,01:07


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.677008,3.677135,0.400000,01:04
1,0.633446,3.656056,0.339130,01:09


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.651349,3.805116,0.382609,01:07


In [44]:
# save the best model

learn.save_encoder('mixed_pubmed_bigtweet_twadr_fold0overlap')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

144
144
[2099, 2099, 1308, 1677, 1932, 359, 840, 803, 1932, 309, 574, 106, 854, 918, 860, 2214, 854, 711, 965, 1442, 106, 803, 744, 1698, 1802, 171, 840, 974, 2027, 1932, 378, 359, 135, 1576, 12, 1811, 2099, 1534, 931, 803, 2099, 1822, 79, 1126, 92, 1935, 1307, 621, 1811, 854, 2099, 1106, 1534, 1106, 135, 1323, 1583, 2182, 1932, 1308, 92, 1932, 1932, 1323, 840, 744, 1948, 635, 2027, 860, 2099, 1811, 309, 1546, 1570, 700, 1534, 854, 978, 1106, 2027, 1038, 620, 2099, 909, 1811, 135, 1698, 1307, 359, 230, 2065, 1534, 903, 378, 1617, 1570, 798, 2065, 1178, 378, 840, 1811, 2099, 2099, 985, 1478, 2097, 2099, 1576, 1411, 2099, 1932, 1639, 1411, 2099, 1811, 1652, 574, 700, 1811, 2027, 860, 375, 268, 1403, 2099, 1811, 935, 87, 170, 106, 574, 309, 2214, 1932, 2097, 1534, 2018, 1698, 1308, 621, 1773, 170]
[1576, 2099, 1698, 1811, 476, 840, 440, 2099, 171, 309, 703, 135, 375, 1326, 860, 574, 2003, 210, 1408, 237, 106, 1576, 700, 1095, 1194, 1307, 239, 935, 833, 1076, 1935, 1008, 2176, 1576, 2088, 

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

22
0.1527777777777778
